In [27]:
# import
import os
from dotenv import load_dotenv

import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.agents import AgentType, initialize_agent, load_tools

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [19]:
# set OpenAI API key
load_dotenv(dotenv_path="../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
# set OpenAI API Key to langchain and set temperature to 0.9
llm = OpenAI(openai_api_key=openai.api_key, temperature=0.9)

In [20]:
llm.predict(f"What would be a good company name for a company for pet supplies?")

'\n\nPawPrints Pet Supplies'

In [21]:
chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to Korean. I love programming.")])

AIMessage(content='나는 프로그래밍을 사랑합니다.', additional_kwargs={}, example=False)

In [22]:
# LLMs
# "text in, text out"
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [23]:
# Chat Models
# interface where "chat messages" are the inputs and outputs.
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [24]:
# Agent with LLMs
# The language model we're going to use to control the agent.
llm = OpenAI(temperature=0)
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# Let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 71.6ºf @2:35 PM Low: 53.6ºf @4:45 AM Approx.
Thought: I now need to use the calculator to raise 71.6 to the .023 power
Action: Calculator
Action Input: 71.6^.023
Observation: Answer: 1.1032222177938817
Thought: I now know the final answer
Final Answer: 1.1032222177938817

> Finished chain.


'1.1032222177938817'

In [25]:
# Agent with Chat Models
# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to search for Olivia Wilde's boyfriend first and then calculate his current age raised to the 0.23 power.

Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```


Observation: Sudeikis and Wilde's relationship ended in November 2020. Wilde was publicly served with court documents regarding child custody while she was presenting Don't Worry Darling at CinemaCon 2022. In January 2021, Wilde began dating singer Harry Styles after meeting during the filming of Don't Worry Darling.
Thought:I found information about Olivia Wilde's boyfriend. Now I need to calculate his current age raised to the 0.23 power.

Action:
```
{
  "action": "Calculator",
  "action_input": "age ^ 0.23"
}
```

ValueError: LLMMathChain._evaluate("
age**0.23
") raised error: 'age'. Please try again with a valid numerical expression

In [26]:
# Memory with LLMs
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [28]:
# Memory with Chat Models
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "The following is a friendly conversation between a human and an AI. The AI is talkative and "
        "provides lots of specific details from its context. If the AI does not know the answer to a "
        "question, it truthfully says it does not know."
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'